In [134]:
!pip install -Uqq ipdb
!pip uninstall numpy
!pip install numpy==1.19.5

!pip uninstall tensorflow
!pip install tensorflow==2.4.1
# !pip install -U tensorflow
import ipdb
from IPython.display import clear_output 
clear_output()

In [135]:
%pdb on #디버깅 모드

Incorrect argument. Use on/1, off/0, or nothing for a toggle.


In [151]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import tensorflow as tf
import sys
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [137]:
def monte_carlo_paths(S_0, time_to_expiry, sigma, drift, seed, n_sims, n_timesteps):
    """
    Create random paths of a underlying following a browian geometric motion
    
    input:
    
    S_0 = Spot at t_0
    time_to_experiy = end of the timeseries (last observed time)
    sigma = the volatiltiy (sigma in the geometric brownian motion)
    drift = drift of the process
    n_sims = number of paths to generate
    n_timesteps = numbers of aquidistant time steps 
    
    return:
    
    a (n_timesteps x n_sims x 1) matrix
    """
    if seed > 0:
            np.random.seed(seed)
    stdnorm_random_variates = np.random.randn(n_sims, n_timesteps)
    S = S_0
    dt = time_to_expiry / stdnorm_random_variates.shape[1]
    r = drift
    # See Advanced Monte Carlo methods for barrier and related exotic options by Emmanuel Gobet
    S_T = S * np.cumprod(np.exp((r-sigma**2/2)*dt+sigma*np.sqrt(dt)*stdnorm_random_variates), axis=1)
    return np.reshape(np.transpose(np.c_[np.ones(n_sims)*S_0, S_T]), (n_timesteps+1, n_sims, 1))

In [138]:
S_0 = 100
K = 100
r = 0
vol = 0.2
T = 1/12
timesteps = 30
seed = 42
n_sims = 10000

paths_train = monte_carlo_paths(S_0,T,vol,r,seed,n_sims,timesteps)

In [139]:
STOCK_GRAPH = paths_train
#print(STOCK_GRAPH.shape) # (31,50000,1) 31일간 시나리오를 50000개 갖고 있다.

STOCK_T = np.reshape(STOCK_GRAPH[-1,:,0],(-1,1))
print(STOCK_T.shape)
STOCK_dS = STOCK_GRAPH[1:,:,0] - STOCK_GRAPH[0:-1,:,0]
STOCK_dS = np.swapaxes(STOCK_dS,0,1)
#STOCK_dS = STOCK_dS.astype(dtype='float64')
STOCK_K = np.full((n_sims,1),100)
print(STOCK_K.shape,STOCK_dS.shape)
STOCK_GRAPH = tf.unstack(STOCK_GRAPH[:-1,:,:],axis=0)
STOCK_GRAPH = tf.stack(STOCK_GRAPH)
STOCK_GRAPH = np.swapaxes(STOCK_GRAPH,0,1)
#STOCK_GRAPH = np.reshape(STOCK_GRAPH,(30,10000))
#print(STOCK_GRAPH.shape)


print(STOCK_GRAPH.shape)

(10000, 1)
(10000, 1) (10000, 30)
(10000, 30, 1)


In [140]:
#@title <font color='Blue'>**Set Value**</font>
batch_size =  100
features = 1
K = 100
alpha = 0.50
num_epoch = 10

_paths_train = np.swapaxes(paths_train,0,1)
train_set = np.array(np.split(_paths_train, int(n_sims/batch_size)))

In [160]:
#@title <font color='Blue'>**Set Model**</font>
class deep_lstm_model(tf.keras.Model):

  def __init__(self):
    super(deep_lstm_model, self).__init__()
    self.lstm_layer = tf.keras.layers.RNN(tf.keras.layers.LSTMCell(4),return_state=True)
    self.dense_layer = tf.keras.layers.Dense(30,activation='tanh')
  

  def call(self, _S_input, _K, _alpha):

    S_input = _S_input #주가 데이터
    K = _K # Strike 가격
    alpha = _alpha # CVaR alpha

    ipdb.set_trace() # 디버깅 지점
    S_T = S_input[-1,:,0] # T(만기일) 때 주가 가격
    dS = S_input[1:,:,0] - S_input[0:-1, :, 0] # 주가 변동 가격
    S_t = tf.unstack(S_input[:-1, :, :], axis=0) #마지막 날짜 제거
    input = tf.experimental.numpy.swapaxes(S_t,0,1)
    # input = np.swapaxes(S_t,0,1) #배열 축 변경

    output , state_c, state_h = self.lstm_layer(input)
    output = self.dense_layer(output)
    output = tf.keras.layers.Dense(1)(output)
    # output = np.array(output.numpy())

    strategy = np.swapaxes(output,0,1)
    option = np.maximum(S_T - K, 0)
    
    Hedging_PnL = -option + np.sum(dS * strategy, axis=0)
    Hedging_PnL_Paths = -option + dS*strategy

    find_k = int((1-alpha)*batch_size)
    top_k = np.partition(Hedging_PnL, -find_k)[-find_k:]
    CVaR = np.mean(top_k)
    print(CVaR)
    return CVaR

In [161]:
#@title <font color='Blue'>**Set Train**</font>
# 최적화를 위한 Adam 옵티마이저를 정의
learning_rate = 0.001
optimizer = tf.optimizers.Adam(learning_rate)

# 최적화를 위한 function을 정의합니다.
@tf.function
def train_step(model, S, K, alpha):
  with tf.GradientTape() as tape:
    CVaR = model(S,K,alpha)
  gradients = tape.gradient(CVaR, model.trainable_variables)
  optimizer.minimize(CVaR, model.trainable_variables)
  return CVaR

In [ ]:
ne#@title <font color='Blue'>**Training**</font>
model = deep_lstm_model()

# 지정된 횟수만큼 최적화를 수행합니다.
for epoch in range(num_epoch):
  # 모든 배치들에 대해서 최적화를 수행합니다.
  for batch_x in train_set:
    # 옵티마이저를 실행해서 파라마터들을 업데이트합니다.
    batch_x = np.swapaxes(batch_x,0,1)
    CVaR = train_step(model, batch_x, K, alpha)
  # 지정된 epoch마다 학습결과를 출력합니다.
  print("반복(Epoch): %d, CVaR: %f" % ((epoch+1), CVaR))

> /tmp/__autograph_generated_fileh3o4xf5m.py(14)tf__call()
     13                 ag__.ld(ipdb).set_trace()
---> 14                 S_T = ag__.ld(S_input)[(- 1), :, 0]
     15                 dS = (ag__.ld(S_input)[1:, :, 0] - ag__.ld(S_input)[0:(- 1), :, 0])

ipdb> next
> /tmp/__autograph_generated_fileh3o4xf5m.py(15)tf__call()
     14                 S_T = ag__.ld(S_input)[(- 1), :, 0]
---> 15                 dS = (ag__.ld(S_input)[1:, :, 0] - ag__.ld(S_input)[0:(- 1), :, 0])
     16                 S_t = ag__.converted_call(ag__.ld(tf).unstack, (ag__.ld(S_input)[:(- 1), :, :],), dict(axis=0), fscope)

ipdb> next
> /tmp/__autograph_generated_fileh3o4xf5m.py(16)tf__call()
     15                 dS = (ag__.ld(S_input)[1:, :, 0] - ag__.ld(S_input)[0:(- 1), :, 0])
---> 16                 S_t = ag__.converted_call(ag__.ld(tf).unstack, (ag__.ld(S_input)[:(- 1), :, :],), dict(axis=0), fscope)
     17                 input = ag__.converted_call(ag__.ld(tf).experimental.numpy.swapaxes, (ag_